In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 

24/03/03 18:25:18 WARN Utils: Your hostname, razer-demian resolves to a loopback address: 127.0.1.1; using 192.168.178.51 instead (on interface enxa0cec8e9c005)
24/03/03 18:25:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/03 18:25:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# default setting
rdd_par = spark.sparkContext.parallelize(dataset_name, 5)

In [ ]:
# with partition argument of 10
# Spark’s default is to partition the text file in 128 MB blocks
rdd_txt = spark.sparkContext.textFile("file_name.txt", 10)

In [ ]:
rdd_txt.getNumPartitions()
# output: 10

In [ ]:
# To end the Spark session
spark.stop()


In [2]:
# Transformations
add_one = lambda x: x+1 # apply x+1 to x
print(add_one(10)) # this will output 11

11


In [4]:
from pyspark.sql import SparkSession
student_data = [("Chris",1523,0.72,"CA"),
                ("Jake", 1555,0.83,"NY"),
                ("Cody", 1439,0.92,"CA"),
                ("Lisa",1442,0.81,"FL"),
                ("Daniel",1600,0.88,"TX"),
                ("Kelvin",1382,0.99,"FL"),
                ("Nancy",1442,0.74,"TX"),
                ("Pavel",1599,0.82,"NY"),
                ("Josh",1482,0.78,"CA"),
                ("Cynthia",1582,0.94,"CA")]
spark = SparkSession.builder.getOrCreate()
student_rdd = spark.sparkContext.parallelize(student_data)

In [5]:
# map transformation
rdd_transformation = student_rdd.map(lambda x: (x[0], x[1], x[2] * 100, x[3]))

# confirm transformation is correct
rdd_transformation.collect()

[('Chris', 1523, 72.0, 'CA'),
 ('Jake', 1555, 83.0, 'NY'),
 ('Cody', 1439, 92.0, 'CA'),
 ('Lisa', 1442, 81.0, 'FL'),
 ('Daniel', 1600, 88.0, 'TX'),
 ('Kelvin', 1382, 99.0, 'FL'),
 ('Nancy', 1442, 74.0, 'TX'),
 ('Pavel', 1599, 82.0, 'NY'),
 ('Josh', 1482, 78.0, 'CA'),
 ('Cynthia', 1582, 94.0, 'CA')]

In [6]:
# filter transformation
rdd_filtered = rdd_transformation.filter(lambda x: x[2] > 80)

# confirm transformation is correct
rdd_filtered.collect()

[('Jake', 1555, 83.0, 'NY'),
 ('Cody', 1439, 92.0, 'CA'),
 ('Lisa', 1442, 81.0, 'FL'),
 ('Daniel', 1600, 88.0, 'TX'),
 ('Kelvin', 1382, 99.0, 'FL'),
 ('Pavel', 1599, 82.0, 'NY'),
 ('Cynthia', 1582, 94.0, 'CA')]

In [7]:
rdd_transformation.take(5)

[('Chris', 1523, 72.0, 'CA'),
 ('Jake', 1555, 83.0, 'NY'),
 ('Cody', 1439, 92.0, 'CA'),
 ('Lisa', 1442, 81.0, 'FL'),
 ('Daniel', 1600, 88.0, 'TX')]

In [8]:
# reduce transformation
sum_gpa = rdd_transformation.map(lambda x: x[2]).reduce(lambda x,y: x+y)

# view the sum
sum_gpa

843.0

In [9]:
sum_gpa / rdd_transformation.count()

84.3

#### Associative and Commutative Properties

In [10]:
from pyspark.sql import SparkSession
spark1 = SparkSession.builder.getOrCreate()

In [11]:
data = [1,2,3,4,5]
for i in range(1,5):
    rdd = spark1.sparkContext.parallelize(data, i)
    print('partition: ', rdd.glom().collect())
    print('addition: ', rdd.reduce(lambda a,b: a+b))

partition:  [[1, 2, 3, 4, 5]]
addition:  15
partition:  [[1, 2], [3, 4, 5]]
addition:  15
partition:  [[1], [2, 3], [4, 5]]
addition:  15
partition:  [[1], [2], [3], [4, 5]]
addition:  15


In [14]:
for i in range(1,5):
    rdd = spark1.sparkContext.parallelize(data, i)
    print('partition: ', rdd.glom().collect())
    print('division: ', rdd.reduce(lambda a,b: a/b))

partition:  [[1, 2, 3, 4, 5]]
division:  0.008333333333333333
partition:  [[1, 2], [3, 4, 5]]
division:  3.3333333333333335
partition:  [[1], [2, 3], [4, 5]]
division:  1.875
partition:  [[1], [2], [3], [4, 5]]
division:  0.20833333333333331


#### Broadcast Variables

In [15]:
# list of states
states = ['FL', 'NY', 'TX', 'CA', 'NY', 'NY', 'FL', 'TX']
# convert to RDD
states_rdd = spark.sparkContext.parallelize(states)

In [16]:
# dictionary of regions
region = {"NY":"East", "CA":"West", "TX":"South", "FL":"South"}

In [17]:
# broadcast region dictionary to nodes
broadcast_var = spark.sparkContext.broadcast(region)
# map regions to states
result = states_rdd.map(lambda x: broadcast_var.value[x])
# view first four results
result.take(4)
# output : [‘South’, ‘East’, ‘South’, ‘West’]

['South', 'East', 'South', 'West']

In [18]:
states = {"NY":"New York", "CA":"California", "TX":"Texas", "FL":"Florida"}

In [19]:
broadcastStates = spark.sparkContext.broadcast(states)

# confirm type
type(broadcastStates)

pyspark.broadcast.Broadcast

In [20]:
rdd_broadcast = rdd_transformation.map(lambda x: (x[0],x[1],x[2], broadcastStates.value[x[3]]))

# confirm transformation is correct
rdd_broadcast.collect()

[('Chris', 1523, 72.0, 'California'),
 ('Jake', 1555, 83.0, 'New York'),
 ('Cody', 1439, 92.0, 'California'),
 ('Lisa', 1442, 81.0, 'Florida'),
 ('Daniel', 1600, 88.0, 'Texas'),
 ('Kelvin', 1382, 99.0, 'Florida'),
 ('Nancy', 1442, 74.0, 'Texas'),
 ('Pavel', 1599, 82.0, 'New York'),
 ('Josh', 1482, 78.0, 'California'),
 ('Cynthia', 1582, 94.0, 'California')]

#### Accumulator Variables

In [21]:
sat_1500 = spark.sparkContext.accumulator(0)

# confirm type
type(sat_1500)

pyspark.accumulators.Accumulator

In [22]:
def count_high_sat_score(r):
    if r[1] > 1500: sat_1500.add(1)

# confirm saved as a function
print(count_high_sat_score)

<function count_high_sat_score at 0x7f2371e885e0>


In [23]:
rdd_broadcast.foreach(lambda x: count_high_sat_score(x))

# confirm accumulator worked
print(sat_1500)

5


### PySpark SQL

#### Creating Spark DataFrames

In [1]:
from pyspark.sql import SparkSession

# Create a new SparkSession
spark = SparkSession\
    .builder\
    .config('spark.app.name', 'learning_spark_sql')\
    .getOrCreate()

sample_page_views  = spark.sparkContext.parallelize([
    ["en", "Statue_of_Liberty", "2022-01-01", 263],
    ["en", "Replicas_of_the_Statue_of_Liberty", "2022-01-01", 11],
    ["en", "Statue_of_Lucille_Ball" ,"2022-01-01", 6],
    ["en", "Statue_of_Liberty_National_Monument", "2022-01-01", 4],
    ["en", "Statue_of_Liberty_play"  ,"2022-01-01", 3],  
])

24/03/04 11:20:48 WARN Utils: Your hostname, razer-demian resolves to a loopback address: 127.0.1.1; using 192.168.178.51 instead (on interface enxa0cec8e9c005)
24/03/04 11:20:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/04 11:20:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# RDD to DataFrame
sample_page_views_df = sample_page_views.toDF(
    ['language_code', 'title', 'date', 'count']
)

# show first 5 rows
sample_page_views_df.show(5, truncate=False)

+-------------+-----------------------------------+----------+-----+
|language_code|title                              |date      |count|
+-------------+-----------------------------------+----------+-----+
|en           |Statue_of_Liberty                  |2022-01-01|263  |
|en           |Replicas_of_the_Statue_of_Liberty  |2022-01-01|11   |
|en           |Statue_of_Lucille_Ball             |2022-01-01|6    |
|en           |Statue_of_Liberty_National_Monument|2022-01-01|4    |
|en           |Statue_of_Liberty_play             |2022-01-01|3    |
+-------------+-----------------------------------+----------+-----+



In [3]:
print(type(sample_page_views_df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [5]:
# DataFrame to RDD
sample_page_views_rdd_restored = sample_page_views_df.rdd

# show restored RDD
sample_page_views_rdd_restored.collect()

[Row(language_code='en', title='Statue_of_Liberty', date='2022-01-01', count=263),
 Row(language_code='en', title='Replicas_of_the_Statue_of_Liberty', date='2022-01-01', count=11),
 Row(language_code='en', title='Statue_of_Lucille_Ball', date='2022-01-01', count=6),
 Row(language_code='en', title='Statue_of_Liberty_National_Monument', date='2022-01-01', count=4),
 Row(language_code='en', title='Statue_of_Liberty_play', date='2022-01-01', count=3)]

In [6]:
print(type(sample_page_views_rdd_restored))

<class 'pyspark.rdd.RDD'>


#### Spark DataFrames from External Sources

In [7]:
from pyspark.sql import SparkSession

# Create a new SparkSession
spark = SparkSession\
    .builder\
    .config('spark.app.name', 'learning_spark_sql')\
    .getOrCreate()

In [ ]:
# Create a DataFrame from a CSV file on disk
wiki_uniq_w_schema_df = spark\
    .read\
    .option('header', True)\
    .option('delimiter', ',')\
    .option('inferSchema', True)\
    .csv('wiki_uniq_march_2022.csv')

# show the data types
wiki_uniq_w_schema_df.dtypes

#### Inspecting and Cleaning Data With PySpark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("learning_spark_sql") \
    .getOrCreate()

# read in the Wikipedia unique visitors dataset
uniq_views_df = spark.read\
    .option('header', True) \
    .option('delimiter', ',') \
    .option('inferSchema', True) \
    .csv("wiki_uniq_march_2022.csv")

In [ ]:
# show the schema
uniq_views_df.printSchema()

In [ ]:
# Describe the data frame
uniq_views_df_desc = uniq_views_df.describe()

# show summary
uniq_views_df_desc.show()

In [ ]:
# drop columns
uniq_counts_human_df = uniq_views_df.drop('total_visitor_count','uniq_bot_visitors')

# show the first 5 rows
uniq_counts_human_df.show(5)

In [ ]:
# rename column
uniq_counts_final_df = uniq_counts_human_df.withColumnRenamed('uniq_human_visitors', 'unique_site_visitors')

# show the first 5 rows
uniq_counts_final_df.show(5)

#### Querying PySpark DataFrames

In [ ]:
from pyspark.sql import SparkSession

# Create a New SparkSession
spark = SparkSession \
    .builder \
    .appName("learning_spark_sql") \
    .getOrCreate()

# Read in Wikipedia Unique Visitors Dataset
wiki_uniq_df = spark.read\
    .option('header', True) \
    .option('delimiter', ',') \
    .option('inferSchema', True) \
    .csv("wiki_uniq_march_2022_w_site_type.csv")

In [ ]:
# filter for Arabic site visitors
ar_site_visitors = wiki_uniq_df\
    .filter(wiki_uniq_df.language_code == 'ar')

# show the DataFrame
ar_site_visitors.show()

In [ ]:
# select columns
ar_visitors_slim = wiki_uniq_df\
    .select(['domain', 'uniq_human_visitors'])\
    .filter(wiki_uniq_df.language_code == 'ar')

# show the DataFrame
ar_visitors_slim.show()

In [ ]:
# group by site type and sum unique visitors
top_visitors_site_type = wiki_uniq_df.select(['site_type', 'uniq_human_visitors'])\
    .groupBy('site_type')\
    .sum()\
    .orderBy('sum(uniq_human_visitors)', ascending=False)

# show the DataFrame
top_visitors_site_type.show()

#### Querying PySpark with SQL

In [ ]:
from pyspark.sql import SparkSession

# Create a new SparkSession
spark = SparkSession \
    .builder \
    .appName("learning_spark_sql") \
    .getOrCreate()

# Read in Wikipedia Unique Visitors Dataset
wiki_uniq_df = spark.read\
    .option('header', True) \
    .option('delimiter', ',') \
    .option('inferSchema', True) \
    .csv("wiki_uniq_march_2022_w_site_type.csv")

# Create a temporary view with the DataFrame
# This will allow us to use SQL to query the data
wiki_uniq_df\
    .createOrReplaceTempView('uniq_visitors_march')

In [ ]:
# SQL query to select Arabic site visitors
ar_site_visitors_qry = """
    SELECT * FROM uniq_visitors_march
    WHERE language_code = 'ar';
"""

# show the DataFrame
spark\
    .sql(ar_site_visitors_qry)\
    .show(truncate=False)

In [ ]:
# SQL query to select specific columns and filter for Arabic site visitors
ar_site_visitors_slim_qry = """
    SELECT domain, uniq_human_visitors 
    FROM uniq_visitors_march
    WHERE language_code = 'ar';
"""

# show the DataFrame
spark\
    .sql(ar_site_visitors_slim_qry)\
    .show(truncate=False)

In [ ]:
# SQL query to group by site type and sum unique visitors
site_top_type_qry = """
    SELECT site_type, SUM(uniq_human_visitors) 
    FROM uniq_visitors_march
    GROUP BY site_type
    ORDER BY SUM(uniq_human_visitors) DESC;
"""

# show the DataFrame
spark\
    .sql(site_top_type_qry)\
    .show(truncate=False)

#### Saving PySpark DataFrames

In [ ]:
from pyspark.sql import SparkSession

# Create a new SparkSession
spark = SparkSession\
    .builder\
    .config('spark.app.name', 'learning_spark_sql')\
    .getOrCreate()

# Read in Wikipedia Unique Visitors Dataset
wiki_uniq_df = spark.read\
    .option('header', True) \
    .option('delimiter', ',') \
    .option('inferSchema', True) \
    .csv("wiki_uniq_march_2022.csv")

In [ ]:
# Create a new DataFrame with the domain and unique visitors
uniq_human_visitors_df = wiki_uniq_df\
    .select('domain', 'uniq_human_visitors')

# show the new DataFrame
uniq_human_visitors_df.show()

In [ ]:
# write the DataFrame to a CSV file
uniq_human_visitors_df\
    .write.csv('./results/csv/uniq_human_visitors/',  mode="overwrite")

In [ ]:
# read the CSV file and show the schema
spark.read.csv('./results/csv/uniq_human_visitors/').printSchema()

In [ ]:
# write the DataFrame to a Parquet file
uniq_human_visitors_df\
    .write.parquet('./results/pq/uniq_human_visitors/', mode="overwrite")

In [ ]:
# read the Parquet file and show the schema
# Parquet is a columnar storage format
# Compared to CSV, Parquet is more efficient for reading and writing and inferSchema is not needed
spark.read.parquet('./results/pq/uniq_human_visitors/').printSchema()